In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
import time

# Configuration du driver Selenium 
service = ChromeService(executable_path='/usr/local/bin/chromedriver')
driver = webdriver.Chrome(service=service)

try:
    # URL de la page à scraper
    url = 'https://www.ebay.fr/b/Telephones-mobiles/9355/bn_16576651'
    driver.get(url)

    # Ouvrir un fichier CSV pour écrire les données
    with open('eBaySmartphones.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Écrire les en-têtes
        writer.writerow(['Article', 'Statut', 'Prix (€)', 'Prix de vente initial (€)', 'Frais de livraison (€)', 'Suivi par (personnes)', 'Vendus', 'Lien du produit'])

        # Boucle pour naviguer sur plusieurs pages
        while True:
            try:
                # Attendre que les éléments soient présents
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 's-item__info')))

                # Récupérer le contenu HTML de la page actuelle
                html = driver.page_source
                bs = BeautifulSoup(html, 'html.parser')

                # Scrapper les articles de Téléphones mobiles
                items = bs.find_all('div', {'class': 's-item__info'})
                for item in items:
                    title_element = item.find('h3', {'class': 's-item__title'})
                    title = title_element.get_text() if title_element else 'N/A'

                    price_element = item.find('span', {'class': 's-item__price'})
                    price = price_element.get_text().replace('EUR', '').strip() if price_element else 'N/A'

                    initial_price_element = item.find('span', {'class': 'STRIKETHROUGH'})
                    initial_price = initial_price_element.get_text().replace('EUR', '').strip() if initial_price_element else 'N/A'

                    shipping_element = item.find('span', {'class': 's-item__shipping s-item__logisticsCost'})
                    frais_livraison = shipping_element.get_text().replace('de frais de livraison', '').replace('EUR', '').strip() if shipping_element else '0'

                    status_element = item.find('div', {'class': 's-item__subtitle'})
                    status = status_element.get_text() if status_element else 'N/A'

                    link_element = item.find('a', {'class': 's-item__link'})
                    link = link_element['href'] if link_element else 'N/A'

                    # Extract additional details from the subtitle text
                    text_element = item.find('span', {'class': 'NEGATIVE'})
                    text = text_element.get_text() if text_element else ''
                    followed = 0
                    vendus = 0
                    if "Suivi par" in text:
                        followed = ''.join([word for word in text.split() if word.isdigit()])
                    elif "Il n'en reste plus qu'un" in text:
                        followed = 1
                    if "vendus" in text:
                        vendus = ''.join([word for word in text.split() if word.isdigit()])

                    # Écrire les données dans le fichier CSV
                    writer.writerow([title, status, price, initial_price, frais_livraison, followed, vendus, link])

                # Vérifier si le bouton 'Suivant' est présent et cliquable
                next_button = driver.find_element(By.CSS_SELECTOR, 'a.pagination__next')
                if 'pagination__next--disabled' in next_button.get_attribute('class'):
                    print("Fin de la pagination.")
                    break
                next_button.click()
                time.sleep(3)  # Attendre quelques secondes pour que la page se charge complètement

            except Exception as e:
                print("Erreur lors de la navigation ou du scrapping:", e)
                print("Attente de 10 secondes pour intervention manuelle...")
                time.sleep(10)  # Attendre 10 secondes pour intervention manuelle
                continue

finally:
    # Fermer le driver Selenium
    driver.quit()


Erreur lors de la navigation ou du scrapping: Message: 
Stacktrace:
#0 0x560a06a5ae3a <unknown>
#1 0x560a0674445c <unknown>
#2 0x560a067905b5 <unknown>
#3 0x560a06790671 <unknown>
#4 0x560a067d4f14 <unknown>
#5 0x560a067b34dd <unknown>
#6 0x560a067d22cc <unknown>
#7 0x560a067b3253 <unknown>
#8 0x560a067831c7 <unknown>
#9 0x560a06783b3e <unknown>
#10 0x560a06a2127b <unknown>
#11 0x560a06a25327 <unknown>
#12 0x560a06a0ddae <unknown>
#13 0x560a06a25df2 <unknown>
#14 0x560a069f274f <unknown>
#15 0x560a06a4a128 <unknown>
#16 0x560a06a4a2fb <unknown>
#17 0x560a06a59f6c <unknown>
#18 0x7c3098c94ac3 <unknown>

Attente de 10 secondes pour intervention manuelle...
Erreur lors de la navigation ou du scrapping: Message: 
Stacktrace:
#0 0x560a06a5ae3a <unknown>
#1 0x560a0674445c <unknown>
#2 0x560a067905b5 <unknown>
#3 0x560a06790671 <unknown>
#4 0x560a067d4f14 <unknown>
#5 0x560a067b34dd <unknown>
#6 0x560a067d22cc <unknown>
#7 0x560a067b3253 <unknown>
#8 0x560a067831c7 <unknown>
#9 0x560a06783b3e

KeyboardInterrupt: 